
# **Gurobi 4: Ejercicio adicional**  

## **Introducción**  
En esta práctica, aprenderás a interpretar la solución de un modelo de optimización y analizar sus resultados.  
Usaremos un **problema de optimización de producción en una fábrica de muebles**, donde se busca **maximizar la utilidad** considerando costos y restricciones.  

---
    


## **Descripción del Problema: Producción Óptima en una Fábrica de Muebles** 🏭📦  

Una fábrica produce **tres tipos de muebles**:  
1. **Mesas**  
2. **Sillas**  
3. **Escritorios**  

Cada mueble requiere **madera y metal** para su fabricación y toma cierto **tiempo de producción** en la planta.  
El objetivo es determinar cuántas unidades de cada mueble fabricar para **maximizar la utilidad total**.  

📌 **Datos del problema:**  

- **Recursos disponibles:**  
  - Madera: **500 m²**  
  - Metal: **300 kg**  
  - Tiempo disponible en la planta: **1000 horas**  

- **Requerimientos de producción (por unidad de mueble):**  

| Mueble     | Madera (m²) | Metal (kg) | Tiempo (h) | Precio de Venta (\$) | Costo de Producción (\$) |
|------------|------------|------------|------------|----------------|----------------|
| Mesa       | 5          | 3          | 8          | 200            | 100            |
| Silla      | 2          | 2          | 5          | 120            | 60             |
| Escritorio | 7          | 4          | 10         | 300            | 180            |  

📌 **Función objetivo:**  
Maximizar la utilidad total:  

$$
\max Z = \sum_{i=1}^{n} (P_i - C_i) x_i
$$  

Donde:  
- $P_i$ es el precio de venta del producto $i$.  
- $C_i$ es el costo de producción del producto $i$.  
- $x_i$ es la cantidad de unidades a fabricar de cada producto.  

📌 **Restricciones:**  
1. **Disponibilidad de recursos:**  
   - Uso total de madera ≤ 500 m²  
   - Uso total de metal ≤ 300 kg  
   - Uso total de tiempo en la planta ≤ 1000 horas  
2. **No podemos fabricar cantidades negativas** (restricción de no negatividad).  

---
    

## **Paso 1: Carga de Datos desde un Archivo CSV o Excel**

In [ ]:

# Importar librerías necesarias
import pandas as pd

# Cargar datos desde CSV
df = pd.read_csv("produccion_muebles.csv")

# Mostrar los datos cargados
print("Datos cargados:")
print(df)


Datos cargados:
       Mueble  Madera  Metal  Tiempo  Precio Venta  Costo Producción
0        Mesa       5      3       8           200               100
1       Silla       2      2       5           120                60
2  Escritorio       7      4      10           300               180


## **Paso 2: Definir el Modelo de Optimización en Gurobi**

In [ ]:
# Descargar e instalar Gurobi desde la fuente oficial
!add-apt-repository -y ppa:deadsnakes/ppa
!apt-get update
!apt-get install -y python3.10
!pip install -i https://pypi.gurobi.com gurobipy

Repository: 'deb https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu/ jammy main'
Description:
This PPA contains more recent Python versions packaged for Ubuntu.

Disclaimer: there's no guarantee of timely updates in case of security problems or other issues. If you want to use them in a security-or-otherwise-critical environment (say, on a production server), you do so at your own risk.

Update Note
Please use this repository instead of ppa:fkrull/deadsnakes.

Reporting Issues

Issues can be reported in the master issue tracker at:
https://github.com/deadsnakes/issues/issues

Supported Ubuntu and Python Versions

- Ubuntu 20.04 (focal) Python3.5 - Python3.7, Python3.9 - Python3.13
- Ubuntu 22.04 (jammy) Python3.7 - Python3.9, Python3.11 - Python3.13
- Ubuntu 24.04 (noble) Python3.7 - Python3.11, Python3.13
- Note: Python2.7 (focal, jammy), Python 3.8 (focal), Python 3.10 (jammy), Python3.12 (noble) are not provided by deadsnakes as upstream ubuntu provides those packages.

Why some

In [ ]:
!pip install gurobipy

# Importar la biblioteca
import gurobipy as gp
from gurobipy import GRB

print("Gurobi instalado y listo para usarse.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 96.8 MB/s eta 0:00:00
Gurobi instalado y listo para usarse.


In [ ]:
# Crear el modelo de optimización
modelo = gp.Model("Produccion_Muebles")

# Crear variables de decisión (cuántas unidades fabricar de cada mueble)
x = modelo.addVars(df["Mueble"], vtype=GRB.CONTINUOUS, lb=0, name="x")

# Definir la función objetivo (maximizar la utilidad total)
modelo.setObjective(
    gp.quicksum((df.loc[df["Mueble"] == m, "Precio Venta"].values[0] -
                 df.loc[df["Mueble"] == m, "Costo Producción"].values[0]) * x[m]
                for m in df["Mueble"]), GRB.MAXIMIZE
)

# Restricción: Disponibilidad de madera
modelo.addConstr(
    gp.quicksum(df.loc[df["Mueble"] == m, "Madera"].values[0] * x[m]
                for m in df["Mueble"]) <= 500, "Madera"
)

# Restricción: Disponibilidad de metal
modelo.addConstr(
    gp.quicksum(df.loc[df["Mueble"] == m, "Metal"].values[0] * x[m]
                for m in df["Mueble"]) <= 300, "Metal"
)

# Restricción: Disponibilidad de tiempo de producción
modelo.addConstr(
    gp.quicksum(df.loc[df["Mueble"] == m, "Tiempo"].values[0] * x[m]
                for m in df["Mueble"]) <= 1000, "Tiempo"
)

# Resolver el modelo
modelo.optimize()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 3 columns and 9 nonzeros
Model fingerprint: 0x184345ba
Coefficient statistics:
  Matrix range     [2e+00, 1e+01]
  Objective range  [6e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve time: 0.01s
Presolved: 3 rows, 3 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.8000000e+32   6.875000e+30   2.800000e+02      0s
       2    1.0000000e+04   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.000000000e+04


## **Paso 3: Interpretar los Resultados**

In [ ]:
# Mostrar la solución óptima
if modelo.status == GRB.OPTIMAL:
    print("Solución óptima encontrada:")
    for m in df["Mueble"]:
        print(f"Fabricar {x[m].x:.2f} unidades de {m}")
    print(f"Utilidad máxima total: {modelo.objVal}")
else:
    print("No se encontró una solución óptima.")


Solución óptima encontrada:
Fabricar 100.00 unidades de Mesa
Fabricar 0.00 unidades de Silla
Fabricar 0.00 unidades de Escritorio
Utilidad máxima total: 10000.0


In [ ]:
# Calcular el uso de cada recurso con la solución óptima
madera_usada = sum(df.loc[df["Mueble"] == m, "Madera"].values[0] * x[m].x for m in df["Mueble"])
metal_usado = sum(df.loc[df["Mueble"] == m, "Metal"].values[0] * x[m].x for m in df["Mueble"])
tiempo_usado = sum(df.loc[df["Mueble"] == m, "Tiempo"].values[0] * x[m].x for m in df["Mueble"])

# Calcular el sobrante de cada recurso
madera_sobrante = 500 - madera_usada
metal_sobrante = 300 - metal_usado
tiempo_sobrante = 1000 - tiempo_usado

# Mostrar resultados
print("\n📊 **Análisis de Uso de Recursos**")
print(f"Madera utilizada: {madera_usada:.2f} m² | Madera sobrante: {madera_sobrante:.2f} m²")
print(f"Metal utilizado: {metal_usado:.2f} kg | Metal sobrante: {metal_sobrante:.2f} kg")
print(f"Tiempo utilizado: {tiempo_usado:.2f} horas | Tiempo sobrante: {tiempo_sobrante:.2f} horas")


📊 **Análisis de Uso de Recursos**
Madera utilizada: 500.00 m² | Madera sobrante: 0.00 m²
Metal utilizado: 300.00 kg | Metal sobrante: 0.00 kg
Tiempo utilizado: 800.00 horas | Tiempo sobrante: 200.00 horas



## **Ejercicios para el Alumno**  
Ahora que has aprendido a resolver un problema de optimización e interpretar los resultados, completa los siguientes ejercicios:  

1. **Modifica los datos del archivo CSV o Excel** para agregar un nuevo tipo de mueble con sus respectivos costos y requerimientos de materiales.  
2. **Cambia la función objetivo** para minimizar costos en lugar de maximizar utilidad.  
3. **Añade una restricción** que limite la producción total de mesas a 50 unidades.  
4. **Explica cómo cambiaría el modelo si se permitiera fabricar productos en fracciones (por ejemplo, 2.5 mesas en lugar de solo valores enteros).**  

✍️ **Instrucciones:**  
- Edita los archivos CSV o Excel y recarga los datos en el modelo.  
- Modifica el código del modelo de optimización para reflejar los cambios.  
- Ejecuta el modelo y analiza los resultados obtenidos.  
    